In [1]:
import phoenix as px

from openinference.semconv.resource import ResourceAttributes
from openinference.instrumentation.dspy import DSPyInstrumentor
# from clank.so.openinference.semconv.resource import ResourceAttributes
# from clank.so-openinference.instrumentation.dspy import DSPyInstrumentor
from opentelemetry import trace as trace_api
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk import trace as trace_sdk
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace.export import SimpleSpanProcessor
from openinference.semconv.trace import SpanAttributes

endpoint = "http://127.0.0.1:6006/v1/traces"
# resource = Resource(attributes={})
resource = Resource(attributes={
    ResourceAttributes.PROJECT_NAME: 'Span-test'
})
tracer_provider = trace_sdk.TracerProvider(resource=resource)
span_otlp_exporter = OTLPSpanExporter(endpoint=endpoint)
tracer_provider.add_span_processor(SimpleSpanProcessor(span_exporter=span_otlp_exporter))
trace_api.set_tracer_provider(tracer_provider=tracer_provider)
DSPyInstrumentor().instrument()

In [2]:
import os
os.environ["LITELLM_LOG"] = "DEBUG"

In [3]:
# %load_ext autoreload
# %autoreload 2
# import sys; sys.path.append('/future/u/okhattab/repos/public/stanfordnlp/dspy')

import dspy
from dspy.evaluate import Evaluate
from dspy.datasets.gsm8k import GSM8K, gsm8k_metric
from dspy.teleprompt import BootstrapFewShotWithRandomSearch
from dspy.modeling import JSONBackend
from dspy.modeling import TextBackend

In [4]:

import openai
client = openai.OpenAI(
    api_key="sk-1234",
    base_url="http://localhost:4000/"
)

# response = client.chat.completions.create(
#     model="llama3:70b", # model to send to the proxy
#     messages = [
#         {
#             "role": "user",
#             "content": "this is a test request, write a short poem"
#         }
#     ]
# )

# print(response)

In [5]:
gms8k = GSM8K()

# backend = JSONBackend(model="gpt-3.5-turbo", params={"max_tokens": 500, "temperature": 0.5, "num_retries": 5})
backend = JSONBackend(model="ollama/llama3:70b", params={"max_tokens": 500, "temperature": 0.1, "num_retries": 5})#, "response_format": {"type": "json_object"}})


# backend = TextBackend(model="ollama/llama3:70b", params={"max_tokens": 500, "temperature": 0.0, "num_retries": 5})#, "response_format": {"type": "json_object"}})

dspy.settings.configure(backend=backend)

trainset, devset = gms8k.train, gms8k.dev

100%|██████████| 1319/1319 [00:00<00:00, 30315.40it/s]


In [6]:
NUM_THREADS = 16
evaluate = Evaluate(devset=devset[:], metric=gsm8k_metric, num_threads=NUM_THREADS, display_progress=True, display_table=0)

In [7]:
class CoT(dspy.Module):
    def __init__(self):
        super().__init__()
        self.prog = dspy.ChainOfThought("question -> answer")
    
    def forward(self, question):
        return self.prog(question=question)

In [8]:
RUN_FROM_SCRATCH = True

if RUN_FROM_SCRATCH:
    # config = dict(max_bootstrapped_demos=5, max_labeled_demos=8, num_candidate_programs=5, num_threads=NUM_THREADS)
    config = dict(max_bootstrapped_demos=2, max_labeled_demos=2, num_candidate_programs=2, num_threads=NUM_THREADS)
    teleprompter = BootstrapFewShotWithRandomSearch(metric=gsm8k_metric, **config)
    cot_bs = teleprompter.compile(CoT(), trainset=trainset, valset=devset)
    # cot_bs.save('turbo_8_8_10_gsm8k_200_300.json')
else:
    cot_bs = CoT()
    cot_bs.load('turbo_8_8_10_gsm8k_200_300.json')

Invalid text provided for JSON extraction: assistant
  0%|          | 0/300 [00:00<?, ?it/s]Invalid text provided for JSON extraction: ", than}, one " is the many " with }, have}"] bookR How " than does a the with one is does the is than more  with one HowR"] one is " is book first ?" does  haveassistant pages the 
Invalid text provided for JSON extraction: 33

333} of3 Tr less3333}33 times three}3

33 less less do than do3 Tr doassistant333 the3 thanassistant3 eggs

?"3 many3 times eggs three Tr of3 the} many
Invalid text provided for JSON extraction: 
Invalid text provided for JSON extraction: "]} as up in to  of40 is

 valid and as up"] in  she A up to them A she number total."40 in up print up total quality is print of is as." up calculate
2024-06-25T04:35:59.484371Z [error    ] Error for example in dev set: 		 Generation failed, recursively attempts to complete did not succeed. [dspy.evaluate.evaluate] filename=evaluate.py lineno=180
2024-06-25T04:35:59.486497Z [error    ] Error f

UnboundLocalError: cannot access local variable 'completion' where it is not associated with a value

In [ ]:
cot_bs.save('turbo_8_8_10_gsm8k_200_300.json')

In [ ]:
evaluate(cot_bs, devset=devset[:])

In [ ]:
print(backend.history[-1].prompt.to_str())